In [ ]:
# !pip install pyshp

In [1]:
import pandas as pd
import numpy
import matplotlib 
%matplotlib inline
import shapefile
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [2]:
data=pd.read_csv('Group017.csv')



.head()

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,Landsize,Age,Lattitude,Longtitude
0,ID275585343,15 Kwinana Ct,3,h,1223500.0,22/08/2016,2.0,2.0,721.0,39.0,-37.75800,145.04080
1,ID702339222,12A Almond St,2,h,997000.0,29/04/2017,1.0,2.0,311.0,94.0,-37.88950,145.02290
2,ID784956327,4/16 Hotham Gr,2,u,660000.0,12/11/2016,1.0,0.0,477.0,49.0,-37.88040,144.99690
3,ID219098083,48 Sydney St,3,h,840000.0,27/11/2016,1.0,0.0,697.0,99.0,-37.78090,144.82280
4,ID292446805,16 Hodgson St,5,h,3450000.0,26/08/2017,2.0,2.0,668.0,13.0,-37.80795,145.01474


In [3]:
sf = shapefile.Reader("./supplementary_data/vic_suburb_boundary/VIC_LOCALITY_POLYGON_shp") # note, no suffix, all 3 files are used
recs = sf.records()
shapes = sf.shapes()

In [4]:
# for x in geometry_from_df:
#     if(x in space_data.geometry):
#         print(x)

In [7]:
def suburbfinder(x):  # function to find suburb
    try:
        latitude = x["Lattitude"]
        longitude = x["Longtitude"]
        for x, y in enumerate(shapes):
            p = Point(longitude, latitude)  # make a point from co ordinates 
            polygon = Polygon(y.points) ## look for that point in polygon
            if polygon.contains(p): ## if present, return its corresponding record 
                #print(list(recs[idx])[6])
                return list(recs[x])[6]
            
        
    except ValueError:  ## else return "not available"
        return "not available"


In [8]:
data["Suburb"] = data.apply(suburbfinder, axis = 1)

In [9]:
data.head()

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,Landsize,Age,Lattitude,Longtitude,Suburb
0,ID275585343,15 Kwinana Ct,3,h,1223500.0,22/08/2016,2.0,2.0,721.0,39.0,-37.75800,145.04080,IVANHOE
1,ID702339222,12A Almond St,2,h,997000.0,29/04/2017,1.0,2.0,311.0,94.0,-37.88950,145.02290,CAULFIELD SOUTH
2,ID784956327,4/16 Hotham Gr,2,u,660000.0,12/11/2016,1.0,0.0,477.0,49.0,-37.88040,144.99690,RIPPONLEA
3,ID219098083,48 Sydney St,3,h,840000.0,27/11/2016,1.0,0.0,697.0,99.0,-37.78090,144.82280,ALBION
4,ID292446805,16 Hodgson St,5,h,3450000.0,26/08/2017,2.0,2.0,668.0,13.0,-37.80795,145.01474,KEW


### Trains

In [39]:
file_name_list = ["routes", "trips", "calendar", "stop_times", "stops"]
file_list = dict()

In [40]:
directory = './supplementary_data/gtfs/1/'

for file in files_list:
    
    filename = directory + file + '.txt'
    
    file_list[file] = pd.read_csv(filename,sep = ',')
#     file_list[filename] = file.readlines()


In [43]:
directory = './supplementary_data/gtfs/2/'

for file in file_list:
    
    filename = directory + file + '.txt'
    
    df = pd.read_csv(filename,sep = ',')
    file_list[file].append(df)

In [48]:
for file in file_list:
    print(file)
    print(file_list[file].head())

routes
        route_id  agency_id route_short_name  \
0  1-V08-A-mjp-1        NaN              NaN   
1  1-V08-B-mjp-1        NaN              NaN   
2  1-V08-C-mjp-1        NaN              NaN   
3  1-V08-D-mjp-1        NaN              NaN   
4  1-V08-E-mjp-1        NaN              NaN   

                               route_long_name  route_type route_color  \
0  Bairnsdale - Melbourne Via Sale & Traralgon           2      8F1A95   
1  Bairnsdale - Melbourne Via Sale & Traralgon           2      8F1A95   
2  Bairnsdale - Melbourne Via Sale & Traralgon           2      8F1A95   
3  Bairnsdale - Melbourne Via Sale & Traralgon           2      8F1A95   
4  Bairnsdale - Melbourne Via Sale & Traralgon           2      8F1A95   

  route_text_color  
0           FFFFFF  
1           FFFFFF  
2           FFFFFF  
3           FFFFFF  
4           FFFFFF  
trips
        route_id service_id                    trip_id           shape_id  \
0  1-V08-G-mjp-1         T0  1051.T0.1-V08-G-mjp-1

In [52]:
routes_trips = pd.merge(file_list['routes'], file_list['trips'], on = 'route_id')

In [54]:
routes_trips_calendar = pd.merge(file_list['calendar'], routes_trips, on = 'service_id')

In [55]:
routes_trips_calendar_stoptimes = pd.merge(routes_trips_calendar,file_list['stop_times'], on = 'trip_id')

In [56]:
merged_df = pd.merge(routes_trips_calendar_stoptimes, file_list['stops'], on = 'stop_id')

In [57]:
merged_df.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,...,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,stop_name,stop_lat,stop_lon
0,UR,0,0,0,1,0,0,0,20190509,20190509,...,06:08:00,20291,1,NaN,0,0,0.000000,Bairnsdale Railway Station (Bairnsdale),-37.82872,147.627614
1,UR,0,0,0,1,0,0,0,20190509,20190509,...,12:45:00,20291,1,NaN,0,0,0.000000,Bairnsdale Railway Station (Bairnsdale),-37.82872,147.627614
2,UR,0,0,0,1,0,0,0,20190509,20190509,...,18:18:00,20291,1,NaN,0,0,0.000000,Bairnsdale Railway Station (Bairnsdale),-37.82872,147.627614
3,UR,0,0,0,1,0,0,0,20190509,20190509,...,11:11:00,20291,23,NaN,0,0,276406.889886,Bairnsdale Railway Station (Bairnsdale),-37.82872,147.627614
4,UR,0,0,0,1,0,0,0,20190509,20190509,...,17:07:00,20291,23,NaN,0,0,276406.889886,Bairnsdale Railway Station (Bairnsdale),-37.82872,147.627614
